In [1]:
import dask_awkward as dak
import awkward as ak
import pandas as pd

from distributed import Client
import glob
import os

client =  Client(n_workers=40,  threads_per_worker=1, processes=True, memory_limit='8 GiB')

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32881 instead
  warnings.warn(


In [3]:
inFile = "/eos/purdue/store/user/rasharma/customNanoAOD_Gautschi_v2/UL2017/VBFHToMuMu_M125_TuneCP5_withDipoleRecoil_13TeV-powheg-pythia8/8996A2F7-0207-6D4B-B4AE-FE0C3C4BFEC1_NanoAOD.root"

In [14]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

NanoAODSchema.warn_missing_crossrefs = False

events = NanoEventsFactory.from_root(
    {inFile: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "VBFHToMuMu"},
).events()
# events = events[:10]

In [15]:
muons = events.Muon
nMuons = ak.num(muons)
# events = events[nMuons == 2]
mu1, mu2 = muons[:, 0], muons[:, 1]

# read only 10 events
# events = events[:10]

In [16]:
events.Muon.pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


<Array [[63.9, 52.5], [63.9], ..., [86, 43.9]] type='6000 * var * float32[p...'>

In [17]:
jets = events.Jet
jet_mask = (jets.pt > 30) & (abs(jets.eta) < 4.7)
selected_jets = jets[jet_mask]

In [18]:
events.Jet.pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


<Array [[83.2, 77.8, ..., 15.3, 15.1], ...] type='6000 * var * float32[para...'>

In [19]:
# events = events[ak.num(selected_jets) >= 2]

# events.Jet.pt.compute()

In [20]:
dr = selected_jets.metric_table(muons)

jet_mu_mask = ak.all(dr > 0.4, axis=2)
selected_jets = selected_jets[jet_mu_mask]

# event_mask = ak.num(selected_jets) >= 2

# events = events[ak.num(selected_jets) >= 2 ]
# events = events[event_mask]

# selected_jets = selected_jets[event_mask]


event_mask = (ak.num(muons) == 2) & (ak.num(selected_jets) >= 2)
events        = events[event_mask]
muons         = muons[event_mask]
selected_jets = selected_jets[event_mask]

In [21]:
events = events[50]
events.Jet.pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called),

<Array [78.2, 60.3, 37.4, ..., 21.6, 17.1] type='10 * float32[parameters={"...'>

In [25]:
events.Muon.pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


<Array [69.7, 46.7] type='2 * float32[parameters={"__doc__": "pt"}]'>

In [ ]:
saj = events.SoftActivityJet

saj_Njets = events[f"SoftActivityJetNjets2"]
saj_HT = events[f"SoftActivityJetHT2"]

In [ ]:
saj_Njets.compute()

In [ ]:
saj_HT.compute()

In [ ]:
saj.pt.compute()

In [ ]:
ak.sum(saj.pt, axis=1).compute()

In [ ]:
njets = ak.num(selected_jets, axis=1)
padded_jets = ak.pad_none(selected_jets, 2)
jet1 = padded_jets[:,0]
jet2 = padded_jets[:,1]

In [ ]:
print(f"jets njets: {njets}")
print(f"jets saj.pt: {saj.pt}")
print(f"jets jet1.pt: {jet1.pt}")
print(f"jets jet2.pt: {jet2.pt}")
print(f"jets mu1.pt: {mu1.pt}")
print(f"jets mu2.pt: {mu2.pt}")

In [ ]:
dR_m1 = saj.delta_r(mu1)
dR_m2 = saj.delta_r(mu2)
dR_j1 = saj.delta_r(jet1)
dR_j2 = saj.delta_r(jet2)
dR_m1_filter = ak.fill_none((dR_m1 < 0.4), value=False, axis=None)
dR_m2_filter = ak.fill_none((dR_m2 < 0.4), value=False, axis=None)
dR_j1_filter = ak.fill_none((dR_j1 < 0.4), value=False, axis=None)
dR_j2_filter = ak.fill_none((dR_j2 < 0.4), value=False, axis=None)
print(f"jets dR_m1_filter: {dR_m1_filter}")
print(f"jets dR_m2_filter: {dR_m2_filter}")
print(f"jets dR_j1_filter: {dR_j1_filter}")
print(f"jets dR_j2_filter: {dR_j2_filter}")

In [ ]:
saj_to_remove = dR_m1_filter | dR_m2_filter | dR_j1_filter | dR_j2_filter
saj_to_remove = ak.fill_none(saj_to_remove, value=False)

footprint = saj[(saj_to_remove) & (saj.pt > 2)]
footprint_sumPt = ak.sum(footprint.pt, axis=1)
print(f"jets saj_to_remove: {saj_to_remove}")
# print(f"jets footprint_sumPt: {ak.to_numpy(footprint_sumPt)}")
footprint_sumPt.compute()

In [ ]:
ht_corrected = saj_HT - footprint_sumPt
footprint_njets = ak.num(footprint, axis=1)
corrected_njets = saj_Njets - footprint_njets

# print(f"jets footprint_njets: {ak.to_numpy(footprint_njets)}")
# print(f"jets corrected_njets: {ak.to_numpy(corrected_njets)}")
print(f"jets saj_Njets: {saj_Njets}")